# Map Creation

Merging the MOOC Metrics (`mooc-metrics.csv`) with a map. Merging on country ISO code, as provided by Google Analytics.

In [1]:
import pandas as pd
import geopandas as gpd

### Countries

Read in the countries and slim down to just the population (for adjusted rates), name, and ISO A2.

In [2]:
ne_50m = '../data/maps/ne_50m_admin_0_countries.geojson'
countries = gpd.read_file(ne_50m)

In [3]:
countries.columns

Index(['abbrev', 'abbrev_len', 'adm0_a3', 'adm0_a3_is', 'adm0_a3_un',
       'adm0_a3_us', 'adm0_a3_wb', 'adm0_dif', 'admin', 'brk_a3', 'brk_diff',
       'brk_group', 'brk_name', 'continent', 'economy', 'featurecla',
       'fips_10', 'formal_en', 'formal_fr', 'gdp_md_est', 'gdp_year',
       'geometry', 'geou_dif', 'geounit', 'gu_a3', 'homepart', 'income_grp',
       'iso_a2', 'iso_a3', 'iso_n3', 'labelrank', 'lastcensus', 'level',
       'long_len', 'mapcolor13', 'mapcolor7', 'mapcolor8', 'mapcolor9', 'name',
       'name_alt', 'name_len', 'name_long', 'name_sort', 'note_adm0',
       'note_brk', 'pop_est', 'pop_year', 'postal', 'region_un', 'region_wb',
       'scalerank', 'sov_a3', 'sovereignt', 'su_a3', 'su_dif', 'subregion',
       'subunit', 'tiny', 'type', 'un_a3', 'wb_a2', 'wb_a3', 'wikipedia',
       'woe_id'],
      dtype='object')

In [4]:
country_df = countries[['iso_a2', 'name_sort', 'scalerank', 'pop_est', 'geometry']]
country_df.head()

,iso_a2,name_sort,scalerank,pop_est,geometry
0,AW,Aruba,3,103065.0,"POLYGON ((-69.89912109375 12.45200195312499, -..."
1,AF,Afghanistan,1,28400000.0,"POLYGON ((74.89130859375001 37.231640625, 74.8..."
2,AO,Angola,1,12799293.0,"(POLYGON ((14.19082031250008 -5.8759765625, 14..."
3,AI,Anguilla,1,14436.0,"POLYGON ((-63.001220703125 18.22177734374999, ..."
4,AL,Albania,1,3639453.0,"POLYGON ((20.06396484375 42.547265625, 20.1035..."


### MOOC Metrics

In [5]:
metrics_f = '../data/canonical/mooc-metrics.csv'
metrics_df = pd.read_csv(metrics_f)

print ("{} rows".format(len(metrics_df)))
metrics_df.head()

1071 rows


,year,country,page,new_users,uniq_pg_views,pg_views
0,2017,BD,poetry-and-plays-2017,6975,8331,10151
1,2017,IN,poetry-and-plays-2017,3819,8556,12849
2,2017,NP,poetry-and-plays-2017,1866,2517,3236
3,2017,PK,poetry-and-plays-2017,1372,2171,2916
4,2017,IN,fiction-and-nonfiction-2017,1711,12900,20355


In [6]:
# sanity check to make sure we can merge
list_of_map_countries = country_df['iso_a2'].tolist()
test_country = lambda x: x in list_of_map_countries

for country in metrics_df['country'].unique().tolist():
    if (test_country(country) == False):
        print (country)

XK
ZZ
nan
GF
RE
BQ
YT
MQ


| ISO A-2 | Country |
|---|---|
| `XK` | Temporary for Kosovo |
| `ZZ` | Unknown |
| `nan` | Not available, pandas error |
| `GF` | [French Guiana](https://en.wikipedia.org/wiki/French_Guiana) |
| `RE` | [Reunion](https://en.wikipedia.org/wiki/R%C3%A9union)
| `BQ` | [Bonaire, Sint Eustatius and Saba](https://en.wikipedia.org/wiki/Caribbean_Netherlands) |
| `YT` | [Mayotte](https://en.wikipedia.org/wiki/Mayotte) |
| `MT` | [Martinique](https://en.wikipedia.org/wiki/Martinique) |


From [Wikipedia article on ISO 3166 A-2](https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2)...

<img src="https://upload.wikimedia.org/wikipedia/commons/d/d2/Kingdom_of_the_Netherlands_location_tree.svg"></img>

In [7]:
country_df[country_df.name_sort == 'Kosovo']['iso_a2']

119    -99
Name: iso_a2, dtype: object

In [8]:
bad_isos = ['nan', 'ZZ', 'GF', 'RE', 'BQ', 'YT', 'MQ']

mask_bad_iso = lambda x: (metrics_df['country'].isin(bad_isos))

metrics_df[mask_bad_iso]

,year,country,page,new_users,uniq_pg_views,pg_views
162,2016,ZZ,how-writers-write-fiction-2016,64,87,118
227,2017,ZZ,fiction-and-nonfiction-2017,48,112,210
261,2017,ZZ,poetry-and-plays-2017,31,49,63
438,2016,ZZ,whitman-2016,18,44,56
469,2015,ZZ,how-writers-write-fiction-2015,35,239,315
545,2016,ZZ,how-writers-write-fiction-2015,7,9,10
584,2017,GF,poetry-and-plays-2017,6,6,6
745,2017,RE,poetry-and-plays-2017,4,5,5
788,2016,BQ,how-writers-write-fiction-2015,2,5,5
904,2017,YT,poetry-and-plays-2017,2,2,2


In [9]:
# reassign the ISO_A2 for kosovo

#mask_koso = (metrics_df['country'] == 'XK')
#metrics_df.loc[mask_koso, 'country'] = -99

# Change source to change it easier
mask_koso_src = (country_df['name_sort'] == 'Kosovo')
country_df.loc[mask_koso_src, 'iso_a2'] = 'XK'

C:\Users\rlrson\AppData\Local\conda\conda\envs\gviz\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [10]:
# Drop the rows from the dataframe which are unidentifiable
mask_zz = (metrics_df['country'] == 'ZZ')

metrics_df = metrics_df.loc[~mask_zz]

In [11]:
# keep only the identified countries
mask_good_iso = lambda x: (~metrics_df['country'].isin(bad_isos))

export_df = metrics_df[mask_good_iso]

print ("{} rows".format(len(export_df)))

1057 rows


### Merging

In [12]:
# get rid of antartica
country_df = country_df[country_df.iso_a2 != 'AQ']

# rename the columns
country_df.columns = ['iso', 'country', 'scalerank', 'population', 'geometry']

# drop the scale rank column
country_df.drop('scalerank', axis=1, inplace=True)

In [13]:
country_df

,iso,country,population,geometry
0,AW,Aruba,103065.0,"POLYGON ((-69.89912109375 12.45200195312499, -..."
1,AF,Afghanistan,28400000.0,"POLYGON ((74.89130859375001 37.231640625, 74.8..."
2,AO,Angola,12799293.0,"(POLYGON ((14.19082031250008 -5.8759765625, 14..."
3,AI,Anguilla,14436.0,"POLYGON ((-63.001220703125 18.22177734374999, ..."
4,AL,Albania,3639453.0,"POLYGON ((20.06396484375 42.547265625, 20.1035..."
5,AX,Aland,27153.0,"(POLYGON ((20.611328125 60.04067382812499, 20...."
6,AD,Andorra,83888.0,"POLYGON ((1.7060546875 42.50332031249999, 1.67..."
7,AE,United Arab Emirates,4798491.0,(POLYGON ((53.92783203125001 24.17719726562498...
8,AR,Argentina,40913584.0,(POLYGON ((-64.54916992187498 -54.716210937499...
9,AM,Armenia,2967004.0,(POLYGON ((45.55234375000006 40.61606445312506...


In [14]:
# rename before merge
export_df.rename(columns={'country': 'iso'}, inplace=True)

C:\Users\rlrson\AppData\Local\conda\conda\envs\gviz\lib\site-packages\pandas\core\frame.py:2844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [15]:
rate_df = export_df.merge(country_df.drop('geometry', axis=1), on='iso')
rate_df.head(10)

,year,iso,page,new_users,uniq_pg_views,pg_views,country,population
0,2017,BD,poetry-and-plays-2017,6975,8331,10151,Bangladesh,1.560509e+08
1,2017,BD,fiction-and-nonfiction-2017,108,283,496,Bangladesh,1.560509e+08
2,2016,BD,how-writers-write-fiction-2016,52,80,115,Bangladesh,1.560509e+08
3,2016,BD,flash-write-2016,32,140,208,Bangladesh,1.560509e+08
4,2015,BD,how-writers-write-fiction-2015,29,165,232,Bangladesh,1.560509e+08
5,2016,BD,how-writers-write-fiction-2015,10,15,16,Bangladesh,1.560509e+08
6,2016,BD,whitman-2016,10,12,12,Bangladesh,1.560509e+08
7,2017,BD,how-writers-write-fiction-2016,2,2,2,Bangladesh,1.560509e+08
8,2017,IN,poetry-and-plays-2017,3819,8556,12849,India,1.166079e+09
9,2017,IN,fiction-and-nonfiction-2017,1711,12900,20355,India,1.166079e+09


In [16]:
# Calculate population adjusted rates
rate_df['new_users_rate'] = rate_df['new_users'] / rate_df['population']
rate_df['uniq_pg_views_rate'] = rate_df['uniq_pg_views'] / rate_df['population']
rate_df['pg_views_rate'] = rate_df['pg_views'] / rate_df['population']

In [17]:
#rate_df.to_csv('../data/canonical/mooc-metrics-population.csv', index=False)

In [18]:
#country_df.to_file('../data/maps/countries-pop.geojson', driver='GeoJSON')

### Single File

Add the classes as a list into the raw GeoJSON.

In [19]:
import json
country_json_str = country_df.to_json()

In [20]:
country_json = json.loads(country_json_str)
country_json.keys()

dict_keys(['type', 'features'])

In [21]:
country_json['features'][0]

{'geometry': {'coordinates': [[[-69.89912109375, 12.452001953124991],
    [-69.895703125, 12.422998046874994],
    [-69.94218749999999, 12.438525390624989],
    [-70.004150390625, 12.50048828125],
    [-70.06611328125, 12.546972656249991],
    [-70.05087890624999, 12.597070312499994],
    [-70.035107421875, 12.614111328124991],
    [-69.97314453125, 12.567626953125],
    [-69.91181640625, 12.48046875],
    [-69.89912109375, 12.452001953124991]]],
  'type': 'Polygon'},
 'id': '0',
 'properties': {'country': 'Aruba', 'iso': 'AW', 'population': 103065.0},
 'type': 'Feature'}

In [22]:
# each feature is saved in a features list.
# we can use a simple list comprehension if we can use the ISO as a lookup key

# create the lookup for said key


In [23]:
rate_df.drop(['country', 'population', 'new_users_rate','uniq_pg_views_rate','pg_views_rate'], 
             axis=1, inplace=True)

In [24]:
list_of_classes = rate_df.to_dict('records')

iso_lookup = {}

for record in list_of_classes:
    iso = record['iso']
    if (iso in iso_lookup):
        iso_lookup[iso].append(record)
    else:
        iso_lookup[iso] = []
        iso_lookup[iso].append(record)

In [25]:
for country in country_json['features']:
    iso = country['properties']['iso']
    
    if (iso in iso_lookup):
        #print (iso, len(iso_lookup[iso]))
        country['properties']['classes'] = iso_lookup[iso]
    else:
        country['properties']['classes'] = []

In [26]:
# calculate the overall for each country
for country in country_json['features']:
    props = country['properties']
    
    for attr in ['new_users', 'uniq_pg_views', 'pg_views']:
        key = 'tot_' + attr
        props[key] = sum(map(lambda x: x[attr], props['classes']))

In [27]:
with open('../data/canonical/country-classes.geojson', 'w') as outfile:
    json.dump(country_json, outfile)